In [1]:
import sys

sys.path.append('/home/z/PycharmProjects/DL_base')
print(sys.path)


['/home/z/PycharmProjects/DL_base/MAIN/multi_input', '/home/z/anaconda3/lib/python310.zip', '/home/z/anaconda3/lib/python3.10', '/home/z/anaconda3/lib/python3.10/lib-dynload', '', '/home/z/.local/lib/python3.10/site-packages', '/home/z/anaconda3/lib/python3.10/site-packages', '/home/z/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/home/z/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg']
['/home/z/PycharmProjects/DL_base/MAIN/multi_input', '/home/z/anaconda3/lib/python310.zip', '/home/z/anaconda3/lib/python3.10', '/home/z/anaconda3/lib/python3.10/lib-dynload', '', '/home/z/.local/lib/python3.10/site-packages', '/home/z/anaconda3/lib/python3.10/site-packages', '/home/z/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/home/z/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg', '/home/z/PycharmProjects/DL_base']


In [4]:
from datetime import datetime
from glob import glob
import json
import os

import cv2
from keras import losses, Model, optimizers, metrics
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, DepthwiseConv2D, Dropout, Flatten, Input, Reshape
from keras.losses import SparseCategoricalCrossentropy
# from keras.metrics import MeanSquaredError, SparseCategoricalAccuracy
from keras.utils import losses_utils, plot_model
from livelossplot import PlotLossesKeras, PlotLossesKerasTF
from matplotlib import pyplot as plt
from numpy import interp
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import label_binarize
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.python.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.python.ops.image_ops_impl import *

from model_helper.process.dataset_mkr import *
from model_helper.process.gpu_memory import *
from model_helper.evaluate.eval_function import *
from model_helper.evaluate.model_visualizer import *

In [ ]:
# load tensorboard
%load_ext tensorboard


In [5]:
"""
0 CPU
1 필요한 만큼 메모리를 런타임에 할당
2 GPU에 할당되는 전체 메모리 크기를 제한
"""

set_gpu_memory(1)

1 Physical GPUs, 1 Logical GPUs


2023-05-18 11:57:49.701340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-18 11:57:49.813644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-18 11:57:49.813922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
MIN, MAX = 0, 255
CLASS = 23
CLASS += 1  ## BG CLASS
print(CLASS)

# 1. DATASET

In [ ]:
data_path = f"/media/z/0/MVPC10/DATA/03_PROCESSED"
label_path = f"/media/z/0/MVPC10/CODE/DL_base/OUT/LABEL/PIXEL_LABEL"

file = f"/media/z/0/MVPC10/DATA/LABEL/v2.2_none_zero.csv"
df = pd.read_csv(file)
df = df.iloc[:,0]
print(df)


In [ ]:
def df_to_tensor(df, path0, path1):
    data = []
    label = []
    # data = np.empty([80, 80], dtype=np.float32)
    # label = np.empty([80, 80], dtype=np.float32)
    for i in range(len(df)):
        try:
            data_img = Image.open(f"{path0}/{df.iloc[i]}.png")
            label_img = Image.open(f"{path1}/{df.iloc[i]}.png")
            data.append(list(data_img.getdata()))
            label.append(list(label_img.getdata()))

            # data_img = cv2.imread(f"{path0}/{df.iloc[i]}.png", 0)
            # label_img = cv2.imread(f"{path1}/{df.iloc[i]}.png", 0)
            # data = np.concatenate((data, data_img), axis=0, dtype=np.float32)
            # label = np.concatenate((label, label_img), axis=0, dtype=np.float32)
        except Exception as E:
            print(E)
            pass
        if i%10000 == 0:  print(i)
    return data, label

data_li, label_li = df_to_tensor(df, data_path, label_path)


In [ ]:
data = np.array(data_li, dtype=np.float32)
data = data.reshape(data.shape[0], 80, 80, 1)
print(data.shape)

label = np.array(label_li, dtype=np.float32)
label = label.reshape(label.shape[0], 80, 80, 1)
print(label.shape)

H, W = data.shape[1], data.shape[2]
print(H, W)


In [ ]:
label.dtype


### 1.5 PROCESS

In [ ]:
## ---------------------------------------------------------------- SHUFFLE
# seed = 99
# np.random.seed(seed)
# np.random.shuffle(data)
# # np.random.seed(seed)
# np.random.shuffle(label)

## ---------------------------------------------------------------- LABEL NORMALIZE
norm_label = label  ## NO NORMALIZE

## ---------------------------------------------------------------- DATA NORMALIZE
# print(data[0][0])
norm_data = data/MAX
# print(norm_data[0][0])

## ---------------------------------------------------------------- SPLIT
split1 = int(len(label)*0.96)
X1, X2 = norm_data[:split1], norm_data[split1:]
Y1, Y2 = norm_label[:split1], norm_label[split1:]
## ---------------------------------------------------------------- VAL SPLIT
# split2 = int(len(label)*0.9)
# X1, val_data = X1[:split2], X1[split2:]
# X2, val_label = X2[:split2], X2[split2:]



In [ ]:
print(X1.shape)
print(Y1.shape)
print(X2.shape)
print(Y2.shape)

# 2. TRAIN

In [ ]:
EPOCH = 16
BATCH = 32
# ES = 16
ES = EPOCH//4

## fit
log_path = f"OUT/{MODEL_VERSION}/logs/"+datetime.now().strftime("%Y%m%d-%H%M%S")
es = EarlyStopping(monitor="val_loss", patience=ES, verbose=2, mode='auto')
tensorboard_callback = TensorBoard(log_dir=log_path, histogram_freq=1)

history = model.fit(X1, Y1,
                    validation_split=0.1,
                    # validation_data=(val_data, val_label),
                    batch_size=BATCH,
                    epochs=EPOCH,
                    use_multiprocessing=True,
                    verbose=1,
                    callbacks=[es,
                               tensorboard_callback,
                               PlotLossesKeras(),
                               # PlotLossesKerasTF(),
                               ],
                    )

# 3. EVALUATE

In [ ]:
# conv_layers1 = []
# for i in model.layers:
#     if 'C2' in i.output:
#         print()
#         # conv_layers1.append(i.output)
# conv_layers1
#

In [ ]:
# a = model.get_layer('8_C2').output
# a

In [ ]:
# conv_layer_index = [1, 5, 9, 13]
# conv_layers = [model.layers[i].output for i in conv_layer_index]
# # conv_layers = [i.output for i in model.layers if "C2" in i.name][:1]
# print(conv_layers)

In [ ]:
# visualize_model = Model(model.inputs, conv_layers)
# print(visualize_model.summary())

In [ ]:
# for i in range(20, 100, 10):
#     img = X2[i]
#     re_img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
#     conv_img = visualize_model.predict(re_img)
#     # columns = int(round(np.sqrt(model.shape[1])))
#     # rows = int(round(np.sqrt(model.shape[2])))
#     columns = 8
#     rows = 8
#     for c_img in conv_img:
#         # pos = 1
#         fig = plt.figure(figsize=(12, 12))
#         for i in range(1, columns*rows+1):
#             fig = plt.subplot(rows, columns, i)
#             fig.axis('off')
#             plt.imshow(c_img[:, :, i-1], cmap='gray')
#             # pos += 1
#         plt.show()

In [ ]:
# filter_visualizer(model, data[0])

In [ ]:
# # visual_keras(model, MODEL_VERSION)
# model_visualizer(model, data[0])

In [ ]:
SAVE_MODEL = 0

if SAVE_MODEL == 1:
    model_save_name = f"OUT/{MODEL_VERSION}/model/{datetime.now().strftime('%Y%m%d-%H%M%S')}.h5"
    model.save(model_save_name)

In [ ]:
# # TENSORBOARD == 1
# # if TENSORBOARD == 1:
# #     launch tensorboard @ localhost:6006
# #     %tensorboard --logdir logs/--host localhost --port 6006
# # %tensorboard --logdir={log_path}

In [ ]:
# ## history to DF
# hdf = pd.DataFrame(history.history)
# hdf.keys()
#
# ## plot history
# hdf.plot(figsize=(9, 6), grid=1, xlabel="epoch", label="accuracy")
# plt.ylim([0, 2])
# plt.show()
#

In [ ]:
# SAVE_MODEL_PLOT = 0
# if SAVE_MODEL_PLOT == 1:
#     plot_model(model, to_file=f"OUT/{MODEL_VERSION}/plot/{MODEL_VERSION}_modelplot.png", show_shapes=True, show_layer_names=False, show_layer_activations=True)


In [ ]:
loss, acc = model.evaluate(X2, Y2, verbose=1)

predict = model.predict(X2)


In [ ]:
## ---------------------------------------------------------------- OUPUT CONVERT
predict_0 = np.reshape(predict, (predict.shape[0], predict.shape[1], predict.shape[2], 1))
Y2_0 = np.reshape(Y2, (Y2.shape[0], Y2.shape[1], Y2.shape[2], 1))
print(predict_0.shape)
print(Y2_0.shape)

In [ ]:
SIZE = 12
num = 9

predict_1 = predict_0 * (MAX-CLASS)
predict_1 = predict_1.astype(np.uint8)

Y2_1 = Y2_0 * (MAX-CLASS)
Y2_1 = Y2_1.astype(np.uint8)

for i in range(1, 10):
    print(i)
    predict_2 = cv2.resize(predict_1[i], (W*SIZE, H*SIZE), interpolation=cv2.INTER_NEAREST)
    img = Image.fromarray(predict_2)

    Y2_2 = cv2.resize(Y2_1[i], (W*SIZE, H*SIZE), interpolation=cv2.INTER_NEAREST)
    img1 = Image.fromarray(Y2_2)

    display(img)
    display(img1)


### 3.1 BIAS FINDER

In [ ]:
n = len(predict_0)
F = 0
pp_li = []
for pp in np.arange(-0.0300, 0.1900, 0.01):
    predict1 = predict_0+pp
    for i in range(n):
        diff = abs(predict1[i].round(0).astype(int)-Y2_0[i])
        F += diff
    pp_li.append((pp,F))
    F = 0

for i in pp_li:
    print(i)

### 3.2 ACCURACY

### 3.3 VISUALIZE